In [5]:
import chardet

def detect_encoding(file_path):
    """Функция для определения кодировки файла."""
    with open(file_path, 'rb') as f:
        raw_data = f.read()
    result = chardet.detect(raw_data)
    return result['encoding']

In [6]:
# !pip install fuzzywuzzy # Требуется для Google Colab
import pandas as pd
from fuzzywuzzy import fuzz

# Функция для нормализации названий регионов
def normalize_region_name(name):
    if not name or not isinstance(name, str):  # Игнорируем пустые или некорректные значения
        return None
    name = name.lower() \
        .replace('городской округ', 'округ') \
        .replace('муниципальный район', 'район') \
        .replace('сельское население', '') \
        .replace('городское население', '') \
        .replace('сельское поселение', 'поселение') \
        .replace('-', '').replace('ё', 'е').strip()
    # Убираем лишние пробелы, но оставляем ключевые слова
    return ' '.join(e for e in name.split() if e.isalnum() or e in ['г.', 'округ', 'район', 'поселение'])

# Улучшенная функция для поиска наиболее подходящего совпадения
def find_matching_region(target, candidates):
    # Проверка на точное совпадение
    if target in candidates:
        return target
    # Проверка на частичное совпадение
    for candidate in candidates:
        if target in candidate or candidate in target:
            return candidate
    # Если ничего не найдено, используем fuzzywuzzy (в крайних случаях)
    max_similarity = 0
    best_match = None
    for candidate in candidates:
        similarity = fuzz.token_set_ratio(target, candidate)
        if similarity > 85 and similarity > max_similarity:
            max_similarity = similarity
            best_match = candidate
    return best_match

# Функция для фильтрации данных по указанной области
def filter_data_by_region(data, target_region):
    filtered_data = []
    collecting = False
    for _, row in data.iterrows():
        region = row['region'].lower()
        if region == target_region.lower():  # Начинаем собирать данные при нахождении целевой области
            collecting = True
            continue
        if collecting:
            if region.endswith('область') or region.endswith('край'):  # Берём только районы и городские округа
                break
            else:
                if normalize_region_name(region):
                    filtered_data.append(row.to_dict())
    return pd.DataFrame(filtered_data).set_index('region')

# Загрузка CSV-файлов от пользователя
csv_files = []
while True:
    file_path = input("Введите путь к CSV файлу (или оставьте пустым для завершения): ").strip()
    if not file_path:
        break
    csv_files.append(file_path)

if not csv_files:
    print("Нет загруженных файлов. Программа завершена.")
    exit()

# Чтение данных из всех CSV-файлов
data_frames = []
russian_headers = []  # Список для хранения русских заголовков
for file in csv_files:
    try:
        encoding = detect_encoding(file)
        # Читаем первую строку как английские заголовки
        df = pd.read_csv(file, sep=';', encoding=encoding, header=0)
        df.columns = [col.strip().lower() for col in df.columns]  # Приводим названия столбцов к нижнему регистру

        # Читаем вторую строку как русские заголовки
        with open(file, 'r', encoding=encoding) as f:
            next(f)  # Пропускаем первую строку (английские заголовки)
            russian_header = next(f).strip().split(';')  # Читаем вторую строку (русские заголовки)
            russian_headers.append([col.strip() for col in russian_header])  # Сохраняем русские заголовки

        df['region'] = df['region'].str.strip().str.lower()  # Нормализуем столбец region
        df = df.dropna(subset=['region'])  # Удаляем строки с пустыми регионами
        data_frames.append(df)
    except Exception as e:
        print(f"Ошибка при чтении файла {file}: {e}")

if not data_frames:
    print("Не удалось загрузить данные из файлов. Программа завершена.")
    exit()

# Пользователь выбирает область для фильтрации
target_region = input("Введите название области для фильтрации (например, 'воронежская область'): ").strip().lower()

# Фильтрация данных по выбранной области
filtered_data = []
for df in data_frames:
    filtered_df = filter_data_by_region(df, target_region)
    if not filtered_df.empty:
        filtered_data.append(filtered_df)

if not filtered_data:
    print("Данные для указанной области не найдены. Программа завершена.")
    exit()

# Функция для выбора столбцов пользователем
def select_columns(df, file_name):
    print(f"\nДоступные столбцы в файле '{file_name}': {df.columns.tolist()}")
    selected_columns = input("Введите через запятую названия столбцов, которые хотите добавить: ").strip()
    if not selected_columns:
        return []
    return [col.strip().lower() for col in selected_columns.split(",")]  # Приводим к нижнему регистру

# Создание комбинированной таблицы
combined_data = []

# Выбор столбцов для каждого файла
selected_columns_per_file = []
for j, df in enumerate(filtered_data):
    selected_columns = select_columns(df, csv_files[j])
    selected_columns_per_file.append(selected_columns)

# Список всех уникальных регионов из всех таблиц
all_regions = set()
for df in filtered_data:
    all_regions.update(df.index.map(normalize_region_name))

# Подсчёт количества вхождений каждого имени столбца
column_counts = {}
for selected_columns in selected_columns_per_file:
    for col in selected_columns:
        column_counts[col] = column_counts.get(col, 0) + 1

# Список всех выбранных столбцов с уникальными именами
all_unique_columns = ['region']  # 'region' всегда добавляется как первый столбец
column_names_tracker = {}  # Словарь для отслеживания уникальных названий столбцов
russian_column_names = {}  # Словарь для хранения русских названий столбцов

# Генерация уникальных имен для выбранных столбцов
for j, selected_columns in enumerate(selected_columns_per_file):
    for col in selected_columns:
        original_name = col
        if column_counts[original_name] > 1:  # Если столбец встречается более одного раза
            unique_name = f"{original_name}_{j}"  # Добавляем суффикс с индексом файла
        else:
            unique_name = original_name  # Оставляем имя без суффикса
        all_unique_columns.append(unique_name)
        column_names_tracker[(original_name, j)] = unique_name

        # Сохраняем русское название столбца
        english_headers = [col.strip().lower() for col in data_frames[j].columns]
        russian_index = english_headers.index(original_name)
        russian_name = russian_headers[j][russian_index]
        russian_column_names[unique_name] = russian_name

# Функция для очистки числовых данных от пробелов
def clean_numeric_value(value):
    if isinstance(value, str):  # Если значение — строка
        value = value.replace(' ', '')  # Убираем пробелы
        try:
            # Попытка преобразовать строку в число (целое или с плавающей точкой)
            if '.' in value:
                return float(value)
            else:
                return int(value)
        except ValueError:
            pass  # Если преобразование не удалось, оставляем строку как есть
    return value

# Обработка всех регионов
for region in all_regions:
    combined_row = {'region': region}
    for j, df in enumerate(filtered_data):
        # Нормализация индексов текущего DataFrame
        df.index = df.index.map(normalize_region_name)
        # Поиск совпадения
        match = find_matching_region(region, df.index.tolist())
        if match and match in df.index:
            match_index = df.index.tolist().index(match)
            # Добавляем выбранные столбцы
            for col in selected_columns_per_file[j]:
                original_name = col
                unique_name = column_names_tracker[(original_name, j)]
                if col in df.columns:
                    value = df.iloc[match_index][col]
                    combined_row[unique_name] = clean_numeric_value(value)  # Очистка числовых данных
        else:
            # Если совпадения нет, добавляем пустые значения
            for col in selected_columns_per_file[j]:
                original_name = col
                unique_name = column_names_tracker[(original_name, j)]
                combined_row[unique_name] = None
    combined_data.append(combined_row)

# Преобразование списка словарей в DataFrame
combined_data_df = pd.DataFrame(combined_data)

# Создание DataFrame для русских названий столбцов
russian_names_row = {col: russian_column_names.get(col, "") for col in all_unique_columns}

# Для region берём русское название из первой таблицы
russian_names_row['region'] = russian_headers[0][0]

# Добавление русских названий в комбинированную таблицу
combined_data_df = pd.concat([pd.DataFrame([russian_names_row]), combined_data_df])

# Убедимся, что region остается первым столбцом
combined_data_df = combined_data_df[all_unique_columns]

# Сохранение комбинированной таблицы в CSV
output_file = input("Введите путь для сохранения комбинированной таблицы (например, 'combined_table.csv'): ").strip()
if not output_file:
    output_file = 'combined_table.csv'
try:
    combined_data_df.to_csv(output_file, sep=';', encoding='utf-8-sig', index=False)  # Убираем индекс Pandas
    print(f"Комбинированная таблица успешно сохранена в файл: {output_file}")
except Exception as e:
    print(f"Ошибка при сохранении файла: {e}")


Доступные столбцы в файле 'csv\Popularity2020.csv': ['total_population_2020', 'male_population', 'female_population', 'male_percent', 'female_percent']

Доступные столбцы в файле 'csv\Popularity2010.csv': ['total_population_2010', 'male_population', 'female_population', 'male_percent', 'female_percent']
Комбинированная таблица успешно сохранена в файл: combined_table.csv
